###### Установка и запуск приложения для отметки изображений

In [ ]:
# !git clone https://github.com/tzutalin/labelImg
# !conda install pyqt=5
# !conda install -c anaconda lxml
# !cd labelImg & pyrcc5 -o libs/resources.py resources.qrc
# !python labelImg/labelImg.py

### Создание директорий

In [ ]:
!mkdir Tensorflow\workspace
!cd Tensorflow\workspace &mkdir annotations images\train images\test models\my_ssd_mobnet pre-trained-models

## 0. Установка путей

In [3]:
WORKSPACE_PATH = "Tensorflow/workspace"
APIMODEL_PATH = "Tensorflow/models"
SCRIPT_PATH = "Tensorflow/scripts/generate_tfrecord.py"
LABEL_MAP_PATH = WORKSPACE_PATH + "/annotations/label_map.pbtxt"
TRAIN_RECORD_PATH = WORKSPACE_PATH + "/annotations/train.record"
TEST_RECORD_PATH = WORKSPACE_PATH + "/annotations/test.record"
MODEL_PATH = WORKSPACE_PATH + "/models"
TRAIN_PATH = WORKSPACE_PATH + "/images/train"
TEST_PATH = WORKSPACE_PATH + "/images/test"
CHECKPOINT_PATH = MODEL_PATH + "/my_ssd_mobnet/"
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + "/pre-trained-models"
PRETRAINED_MODEL_CHECKPOINT = PRETRAINED_MODEL_PATH + "/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
PRETRAINED_MODEL_CONFIG = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'
CUSTOM_MODEL_NAME = "my_ssd_mobnet"

## 1. Создание карты меток

In [2]:
labels = [{"name": "Mask", "id": 1}, {"name": "NoMask", "id": 2}]

with open(LABEL_MAP_PATH, "w") as f:
    for label in labels:
        f.write("item {\n" + 
                "\tname: '{name}'\n".format(**label) + 
                "\tid:{id}\n".format(**label) + 
                "}\n")

## 2. Загрузка предварительно обученных моделей TensorFlow

In [3]:
!cd Tensorflow & git clone https://github.com/tensorflow/models

### Установка необходимых пакетов для обнаржения объектов

In [2]:
RESEARCH_PATH = "Tensorflow/models/research"
!cd {RESEARCH_PATH}&protoc object_detection/protos/*.proto --python_out=.
!cd {RESEARCH_PATH}&cp object_detection/packages/tf2/setup.py .
!cd {RESEARCH_PATH}&python -m pip install .

Processing c:\users\squoz\desktop\face-mask_detector\tensorflow\models\research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1664572 sha256=b6e41fa89cbcbcb54b8b36154a66fe341a8d1e20beb9bd2af66be7a9e1be2e77
  Stored in directory: C:\Users\squoz\AppData\Local\Temp\pip-ephem-wheel-cache-yljmb7zg\wheels\8f\ee\22\a5331a2aa3a5332ea65b7da6f4ddb1e06f51659dff164c65be
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


## 3. Создание файлов-записей Tensorflow для обучения и тестирования

In [3]:
!python {SCRIPT_PATH} -x {TRAIN_PATH} -l {LABEL_MAP_PATH} -o {TRAIN_RECORD_PATH}
!python {SCRIPT_PATH} -x {TEST_PATH} -l {LABEL_MAP_PATH} -o {TEST_RECORD_PATH}

Файл TFRecord успешно создан: Tensorflow/workspace/annotations/train.record
Файл TFRecord успешно создан: Tensorflow/workspace/annotations/test.record


## 4. Загрузка и распаковка одноэтапной модели обнаружения объектов SSD-MobileNet V2

In [4]:
!pip install wget
from wget import download
SSD_MobNetV2_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
download(SSD_MobNetV2_URL, out = PRETRAINED_MODEL_PATH)
# Перемещение в папку с архивом и распаковка
!cd {PRETRAINED_MODEL_PATH} & tar -xvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

100% [........................................................................] 20515344 / 20515344ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


## 5. Копирование конфигурации модели в папку для обучения

In [4]:
!cp {PRETRAINED_MODEL_CONFIG} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

## 6. Обновления конфигурации для трансферного обучения

In [5]:
import tensorflow as tf
from google.protobuf.text_format import Merge, MessageToString
from object_detection.protos.pipeline_pb2 import TrainEvalPipelineConfig

Проверка доступности видеокарты

In [6]:
gpu_available = tf.config.list_physical_devices('GPU')
print("Количество доступных видеокарт:", len(gpu_available))

is_cuda_gpu_min_5 = tf.test.is_gpu_available(True, (5,0))
print('Вычислительная мощность CUDA больше 5.0:', is_cuda_gpu_min_5)

Количество доступных видеокарт: 1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Вычислительная мощность CUDA больше 5.0: True


In [10]:
CONFIG_FILE = MODEL_PATH + "/" + CUSTOM_MODEL_NAME + "/pipeline.config"
pipeline_config = TrainEvalPipelineConfig()

with tf.io.gfile.GFile(CONFIG_FILE, "r") as f:
    proto_str = f.read()
    Merge(proto_str, pipeline_config)

Список изменений в конфигурации

In [8]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = (PRETRAINED_MODEL_PATH + "/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0")
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = LABEL_MAP_PATH
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [TRAIN_RECORD_PATH]
pipeline_config.eval_input_reader[0].label_map_path = LABEL_MAP_PATH
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [TEST_RECORD_PATH]

Запись изменений в файл конфигурации

In [9]:
config_text = MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_FILE, "wb") as f:
    f.write(config_text)

## 7. Тренировка модели

Для тренировки модели необходимо запустить выведенную команду в корневой директории проекта и только после завершения тренировки продолжить запуск кода

In [11]:
print(f"python {APIMODEL_PATH}/research/object_detection/model_main_tf2.py " 
      f"--model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} "
      f"--pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config "
      "--num_train_steps=6000")

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=6000


## 8. Загрузка тренировочной модели с чекпоинта

In [12]:
from object_detection.builders.model_builder import build
from object_detection.utils.config_util import get_configs_from_pipeline_file
from os.path import join

Загрузка конфигурации и построение модели обнаружения

In [13]:
configs = get_configs_from_pipeline_file(CONFIG_FILE)
detection_model = build(model_config=configs["model"], is_training=False)

Восстановление точки сохранения

In [14]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(join(CHECKPOINT_PATH, "ckpt-11")).expect_partial()

In [15]:
@tf.function
def detect(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 9. Обнаружение маски в реальном времени

In [16]:
import cv2
from tkinter import Tk
from tkinter.messagebox import askyesno
from vlc import MediaPlayer
from numpy import array, expand_dims, int64
from object_detection.utils.label_map_util import create_category_index_from_labelmap
from object_detection.utils.visualization_utils import visualize_boxes_and_labels_on_image_array

In [17]:
category_index = create_category_index_from_labelmap(LABEL_MAP_PATH)

In [18]:
# Видеозахват веб-камеры. В качестве аргумента указывается ID-устройства, либо путь к медиафайлу
camera = cv2.VideoCapture(0)
# # Получение ширины и высоты окна
# width, height = ( 
#     int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
#     int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)),
# )

root = Tk()
root.withdraw()

snd = MediaPlayer("sounds/sound-2.mp3")

while True:
    # Считывание кадров с камеры
    _, frame = camera.read()
    # Перевод кадров в массив
    image_arr = array(frame)
    # Перевод массива в тензоры
    input_tensor = tf.convert_to_tensor(expand_dims(image_arr, 0), dtype=tf.float32)
    detections = detect(input_tensor)
    num_detections = int(detections.pop("num_detections"))
    detections = {
        key: value[0, :num_detections].numpy() for key, value in detections.items()
    }
    detections["num_detections"] = num_detections
    detections["detection_classes"] = detections["detection_classes"].astype(int64)
        
    # Для отсчёта индекса классов с единицы, вместо нуля
    label_id_offset = 1
    # Копирование оригинального кадра для наложения визуализации
    image_arr_with_detections = image_arr.copy()

    # Визуализация
    visualize_boxes_and_labels_on_image_array(
        image_arr_with_detections,
        detections["detection_boxes"],
        detections["detection_classes"] + label_id_offset,
        detections["detection_scores"],
        category_index,
        # Установка визуализации поверх объекта обнаружения, а не в координатах (0, 0)
        use_normalized_coordinates=True,
        # Максимальное количество визуализированных полей
        max_boxes_to_draw=1,
        # Толщина линий поля
        line_thickness=2,
        # Минимальная отметка точн/ости - 50%
        min_score_thresh=0.5
#         agnostic_mode=False
    )
    
    # Трансляция с камеры и установка размера окна
    cv2.imshow(
        'Обнаружение маски. Для выхода нажмите "q"',
        cv2.resize(image_arr_with_detections, (1000, 750))
    )
    # Проверка на наличие маски
    no_mask = (detections["detection_classes"][0])
#     if no_mask:
#         snd.play()
#     else:
#         snd.stop()
        
    if no_mask:
        prompt = askyesno(
            'Предупреждение',
            'Наденьте маску и нажмите "Yes", чтобы продолжить\nНажмите "No", чтобы выйти'
        ) 
        if prompt == True:
            continue
        elif prompt == False:
            break

    # Привязка завершения цикла обнаружения на клавишу 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

snd.stop()
root.quit()
# Высвобождение камеры
camera.release()
# Закрытие окон OpenCV
cv2.destroyAllWindows()